<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Configuration And Utils </span> </p>

In [ ]:
#!import "../Utils/EqualityComparers"
#!import "../Utils/Queries"

In [ ]:
using Systemorph.Vertex.Pivot.Builder;
using Systemorph.Vertex.Pivot.Reporting.Builder;
using System.Collections.Immutable;

public static DataCubeReportBuilder<IDataCube<TVariable>, TVariable, TVariable ,TVariable> ReportGridOptions<TVariable>(
    this DataCubePivotBuilder<IDataCube<TVariable>, TVariable, TVariable, TVariable> reportBuilder,
    int reportHeight = 700,
    int valueColumnWidth = 250,
    int headerColumnWidth = 250,
    int groupDefaultExpanded = 2)
    => reportBuilder.WithGridOptions(go => go
                                     .WithColumns(cols => cols.Modify("Value",c => c.WithWidth(valueColumnWidth).WithFormat("new Intl.NumberFormat('en',{ minimumFractionDigits:2, maximumFractionDigits:2 }).format(value)")))
                                     .WithRows(rows => rows
                                               .Where(r => !(r.RowGroup.Coordinates.Last() == "NullGroup"))
                                               .Select(r => r with { RowGroup = r.RowGroup with { Coordinates = r.RowGroup.Coordinates.Where(c => c != "NullGroup").ToImmutableList() } })
                                               .ToArray())
                                     .WithAutoGroupColumn(c => c.WithWidth(headerColumnWidth) with { Pinned = "left" })
                                     with { Height = reportHeight, GroupDefaultExpanded = groupDefaultExpanded, OnGridReady = null } );

# Currency methods

In [ ]:
public static double GetCurrencyToGroupFx(Dictionary<string, Dictionary<FxPeriod, double>> exchangeRates, string currency, FxPeriod fxPeriod, string groupCurrency)
{
    if(currency == groupCurrency)
      return 1;

    if(!exchangeRates.TryGetValue(currency, out var currencyToGroup))
      ApplicationMessage.Log(Error.ExchangeRateCurrency, currency);

    if(!currencyToGroup.TryGetValue(fxPeriod, out var currencyToGroupFx))
      ApplicationMessage.Log(Error.ExchangeRateNotFound, currency, fxPeriod.ToString());

    return currencyToGroupFx;
}

# Queries

## Query Report Variables

In [ ]:
public static async Task<ReportVariable[]> QueryReportVariablesSingleScenarioAsync(this IWorkspace workspace, (int Year, int Month, string ReportingNode, string Scenario) args ) {

    await workspace.Partition.SetAsync<PartitionByReportingNode>(new { ReportingNode = args.ReportingNode, Scenario = (string)null}); // TODO: Scenario from args + relaxed query 
    await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(new { ReportingNode = args.ReportingNode, Scenario = args.Scenario, Year = args.Year, Month = args.Month });
    
    var reportVariables = await workspace.Query<GroupOfContract>()
            .Join(workspace.Query<IfrsVariable>(),
                  dn => dn.SystemName,
                  iv => iv.DataNode,
                  (dn, iv) => new ReportVariable {
                      // From Args/Inputs
                      ReportingNode = args.ReportingNode,
                      Scenario = args.Scenario,
                      
                      // From Data Node
                      Portfolio = dn.Portfolio,
                      GroupOfContract = dn.SystemName,
                      
                      FunctionalCurrency = dn.FunctionalCurrency,
                      ContractualCurrency = dn.ContractualCurrency,
                      
                      ValuationApproach = dn.ValuationApproach,
                      OciType = dn.OciType,
                      InitialProfitability = dn.Profitability,
                      LiabilityType = dn.LiabilityType,
                      AnnualCohort = dn.AnnualCohort,
                      LineOfBusiness = dn.LineOfBusiness,
                    
                      // Reinsurance
                      IsReinsurance = dn is GroupOfReinsuranceContract, 
                      Partner = dn.Partner,
                      
                      // From Variable
                      EstimateType = iv.EstimateType,
                      VariableType = iv.AocType,
                      Novelty = iv.Novelty,
                      AmountType = iv.AmountType,
                      EconomicBasis = iv.EconomicBasis,
                      
                      //LIC
                      AccidentYear = iv.AccidentYear == null ? default : (int)iv.AccidentYear,
                      
                      Value = iv.Value })
        .ToArrayAsync();

    await workspace.Partition.SetAsync<PartitionByReportingNode>(null);
    await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(null);
    return reportVariables;
}

In [ ]:
public static async Task<ICollection<ReportVariable>> QueryReportVariablesAsync(this IWorkspace workspace, (int Year, int Month, string ReportingNode, string Scenario) args ) {
    ReportVariable[] reportVariables = new ReportVariable[0];
    if(args.Scenario != null) 
        reportVariables = (await workspace.QueryReportVariablesSingleScenarioAsync((args.Year, args.Month, args.ReportingNode, null)));
    return (await workspace.QueryReportVariablesSingleScenarioAsync((args.Year, args.Month, args.ReportingNode, args.Scenario)))
            .Union(reportVariables.Select(x => x with {Scenario = args.Scenario}), EqualityComparer<ReportVariable>.Instance).ToArray();
}